Downlaod necessary libraries

In [4]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
!pip install --no-deps unsloth
!pip install --no-deps pandas

Looking in indexes: https://download.pytorch.org/whl/cu128
  Using cached bitsandbytes-0.46.0-py3-none-win_amd64.whl.metadata (10 kB)
  Using cached accelerate-1.7.0-py3-none-any.whl.metadata (19 kB)
  Using cached xformers-0.0.29.post3.tar.gz (8.5 MB)


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\fonta\\AppData\\Local\\Temp\\pip-install-jrq2www8\\xformers_28ffc2b184694562b8b0500658d04fcd\\third_party\\flash-attention\\csrc\\composable_kernel\\client_example\\24_grouped_conv_activation\\grouped_convnd_bwd_data_bilinear\\grouped_conv_bwd_data_bilinear_residual_fp16.cpp'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths



  Using cached sentencepiece-0.2.0.tar.gz (2.6 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [48 lines of output]
      Traceback (most recent call last):
        File "C:\Users\fonta\Desktop\Uni\Repo\NLP\Exam\env\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 389, in <module>
          main()
          ~~~~^^
        File "C:\Users\fonta\Desktop\Uni\Repo\NLP\Exam\env\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 373, in main
          json_out["return_val"] = hook(**hook_input["kwargs"])
                                   ~~~~^^^^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\fonta\Desktop\Uni\Repo\NLP\Exam\env\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 143, in get_requires_for_build_wheel
          return hook(config_settings)
        File "C:\Users\fonta\AppData\Local\Temp\pip-build-env-zll_rz4b\overlay\Lib\site-packages\setuptools\build_meta.py",

Import of necessary libraries

In [4]:
import pandas as pd
import re
import torch


print(torch.cuda.is_available())
#from unsloth import FastLanguageModel



True


Dataset clean (only once)

In [6]:
dataset= pd.read_csv("explaination_toxic_conversation_most_toxic_sentences.csv",header=0, index_col=0, encoding='utf-8')
dataset = dataset[dataset['explaination'].str.contains("La frase più tossica è", na=False)]
#add a new column 'toxic' with value 1 for all rows
dataset['toxic'] = 1

# View the first few rows of the dataset with the new 'toxic_phrase' column
print(dataset.loc[:, ['toxic']].head())

dataset.to_csv("filtered_dataset.csv", index=True, encoding='utf-8')

                                   toxic
person_couple                           
Psicopatico e Adulatrice               1
Manipolatore e Dipendente emotiva      1
Persona violenta e Succube             1
Psicopatico e Adulatrice               1
Narcisista e Succube                   1


SET preparation

In [21]:
dataset=pd.read_csv("filtered_dataset.csv",header=0, index_col=0, encoding='utf-8')

#take 70% of the dataset for training and 30% for testing, toxic column is the target
train_size = int(0.7 * len(dataset))
X_train, X_test, y_train, y_test = dataset.iloc[:train_size, :], dataset.iloc[train_size:, :], dataset['toxic_phrase'][:train_size], dataset['toxic_phrase'][train_size:]

#print the shape of every set
print(f"Training set shape: {X_train.shape}, Test set shape: {X_test.shape}")
print(f"Training labels shape: {y_train.shape}, Test labels shape: {y_test.shape}")

Training set shape: (682, 5), Test set shape: (293, 5)
Training labels shape: (682,), Test labels shape: (293,)
